In [1]:
# coding: utf-8
#ifrom __future__ import print_function

from keras.models import Model
from keras.models import load_model
from keras.layers import Input, LSTM,Embedding, Dense
import keras

#from sklearn.model_selection import train_test_split

import numpy as np
import json
import re
#import pydot

import nltk
from nltk.tree import Tree
from nltk.translate.bleu_score import sentence_bleu

####

import logging

from nltk.sem.logic import LogicParser
from nltk.sem.logic import LogicalExpressionException

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [13]:
#bleu evaluation
len_inp = 1500#len(test_input_data)
sum_score = 0
complete = 0
comp_texts = []
for seq_index in range(len_inp-1):
    fname = 'c2l/result'+str(seq_index)+'.txt'
    f = open(fname)
    lines = f.readlines()
    f.close()

    orign_text = lines[0].rstrip()
    if(orign_text[-1]!='.'):
        orign_text = orign_text + '.'
    decoded_sentence = lines[1].rstrip()
    score = sentence_bleu([orign_text],decoded_sentence)
    sum_score += score
    
    if(orign_text == decoded_sentence):
        complete += 1
        comp_texts.append(orign_text)
    
    print(orign_text)
    print(decoded_sentence)
    print(score,"\n")
    
    #f.write(output_texts[seq_index])
    #f.write(decoded_sentence.strip()+'\n')
    #f.close()
    #print('Input sentence:', input_texts[seq_index])
    #if (seq_index%100) == 0 :
    #    print('Decoded sentence:', decoded_sentence)
    #    print('Answer sentence:', output_texts[seq_index])
print('bleu score: ',(sum_score/len_inp))
print('complete: ',complete)

Two women are embracing while holding to go packages.
Two women are holding something and walking.
0.49563485446627453 

The sisters are hugging goodbye while holding to go packages after just eating lunch.
A group of grimacing males is showing a performance before a third player.
0.14225699613496043 

Two woman are holding packages.
Two women are holding a camera.
0.645140998342485 

The men are fighting outside a deli.
A man is a marathon runner.
0.1943554815299873 

Two young children in blue jerseys, one with the number 9 and one with the number 2 are standing on wooden steps in a bathroom and washing their hands in a sink.
Two young people are wearing a red and white and orange bikini pose is.
0.1309479220527187 

Two kids in numbered jerseys wash their hands.
Two kids were wearing orange field on a yellow race.
0.25325897139464854 

Two kids at a ballgame wash their hands.
Two kids were looking up in the sky.
0.31348626641747357 

Two kids in jackets walk to school.
Two girls in 

/usr/local/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)



The child is dancing inside.
Children are dancing in a restaurant.
0.36811442378930576 

A young boy playing in the grass.
A young boy playing in a fountain.
0.6880702830421056 

The boy is playing make believe.
A boy makes a play.
0.3019248782293741 

There is a child in the grass.
There are children in the snow.
0.5448053609858732 

The boy is in the sand.
The boy is in the sand.
1.0 

Two men playing baseball with the one in the black and red jersey running toward base.
Two men are playing with a small girl in the background and a small yellow.
0.4474254085514025 

The man in black and red is about to score the winning run.
The man in black is about to swing a tennis ball.
0.58115580728791 

There is a baseball game going on.
There is a baseball game on his football.
0.6468733035162956 

The man in black and red is sitting in the bleachers.
The man in the black shirt is sitting on a bench.
0.6503109204064274 

A man and a woman are looking at produce on display.
A man and a woman a

/usr/local/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


two people are waiting for a train at a station.
The two people are waiting at the station waiting.
0.6698663020109664 

They expect a train to arrive eventually.
They are going to rain outside.
0.27244773966824 

They are both waiting for a bus.
They are waiting for the bus.
0.753005252831695 

They will both get onto the same train together.
They are going to the side of a town.
0.30337673643104046 

A man in a black shirt is smiling at a woman in a black shirt what a tattoo and a eye brow ring.
A man in a black shirt and black jeans is in a yellow shirt practicing.
0.48751270832463045 

A man is smiling at a woman.
A man is reading a woman outside.
0.5611004026043056 

A man is smiling at his wife.
A man is reading a book at.
0.44259128097970496 

A man is reading a book alone.
A man is reading a book book.
0.806096123702637 

A person wearing white is running towards a ball that was kicked from another person.
A person wearing a yellow helmet is running towards a bar under a large.

They are admiring the beautiful flowers in the garden.
They are doing a great in a public.
0.25870141362869553 

a man is swimming inside of a pool.
The man is swimming The man is his friends.
0.43861355687480014 

the person is in the water playing.
A person is in a crowded building.
0.48132987582133324 

there is a person drowning.
There is a person bike.
0.6221481193503641 

there is a man in a pool.
There is a man in a pool.
0.9573479717381596 

a little girl sitting in a seat.
A little girl is sitting in a baby.
0.7265551147478562 

She is sitting nicely.
she is sitting.
0.5434614713253186 

She is standing on the seat.
She is standing on a couch.
0.6929234193983161 

She is in a chair.
The is is in a show.
0.5426113542926237 

A hispanic fruit market with many different fruits and vegetables in view on a city street with a man passing the store dressed in dark pants and a hoodie.
An older man in a checkered shirt and gray jeans stands on a rock in.
0.11153853485122675 

A fruit m

A father holds his two sleeping children.
A father and son are running.
0.2552877315349576 

A man holds two sleeping children.
A man takes a child for a dog.
0.329701852184083 

People on a concrete area taking pictures.
People are working with a white sheet of rollerskate.
0.3126882888781631 

people take pictures.
People are taking a picture.
0.5325316764898671 

boy sleeps alone.
The boy is sleeping on the bed.
0.26276400987738757 

people posting pics on facebook.
People are working on a mountain mountain.
0.24774545263373765 

Theses people is taking a picture of a pink pig lying on the ground.
A group of people standing outside trying to one in a white sheet.
0.3957160293544623 

The people are throwing a birthday party.
People are eating a birthday party.
0.7133468127458864 

These people are photographing a pig on the ground.
People are going on a tour.
0.27299792454124766 

A group spotted a pig nearby.
The baby is feeding the skateboarder.
0.5610704892147496 

A woman rock-c

A woman is holding a baby by a window.
A woman is holding a baby while a baby.
0.738293248607273 

There is a baby in the picture.
There is a baby in a picture.
0.8457980831074701 

The woman is looking out the window.
A woman looks out into the ocean.
0.49131305839794115 

The woman 's hands are empty.
The woman 's woman 's are.
0.5828375171111605 

A man and woman are sitting at a table in front of a sign that reads IL PORTO and two wine bottles and two wineglasses are on the table.
A man and a woman are seated on a large table looking at a restaurant.
0.2465428139734258 

Two people participate in a wine tasting.
Two people are referencing a hole in a rowboat.
0.4147926680255282 

Two people are ingesting alcohol.
The two people are eating a canoe.
0.5592357668262025 

Two teetotalers prepare for dinner.
Two friends are preparing a meal.
0.3635838644761767 

Two men wrestle on the floor in front of an group of young men.
Two men are on the front of a wood by building a liquid.
0.463

0.23157668035444887 

A man and a woman are jumping in midair and holding hands, a cloudy sky in the background.
A man in a blue sweatshirt is holding a flower up his phone and another.
0.31056093656586536 

The man and woman are physically touching each other.
The men and women are are getting their drink.
0.3886380799415245 

The man and woman are married.
A man and woman are married.
0.8958643779263316 

The man and woman jumped 20 ft into the air.
A man and woman jumps into the ocean.
0.6192539412430093 

Five Women, wearing beautiful, colorful saris, in a circle, dancing with their hands raised and touching.
Two women in street performers are wearing white outfits are dancing.
0.24458703250580233 

There are five women close to each other.
Five women are all near each other.
0.5893509555856866 

The women are friends.
The women are friends.
1.0 

The women have their hands by their side.
The woman has a camera as a group of people.
0.24308003014675808 

Five little boys are playin

A child sleeps under a tree during a tree.
0.2508916472831742 

A baby.
A trainer and his pooch.
0.29176300840900804 

a baby in crib.
A baby in a diner.
0.5085991394194219 

A toddler slept.
A toddler is swimming.
0.44231933624679737 

Balding man in gray sweater plays with his dog.
A man in a gray sweater is playing with his son.
0.7196463944996859 

A man puts the sweater he knitted on a dog.
The man is using the box of a jet washer on the woods.
0.26357149648074785 

An older man is throwing a stick for his dog.
The older man is playing the ball for the ball.
0.44762672628271677 

A man is with an animal.
A man is with toys.
0.5574621193620891 

The small girl is sitting in a harness, strapped to a tether, while others watch.
A smiling and a girl are walking together in the water near a green balls.
0.31240252786105455 

There is a girl in a harness.
The girl in the uniform is related.
0.38017154436471184 

A little girl is riding a bicycle on a tightrope.
A little girl rides on a 

In [12]:
print(comp_texts)

['A man is wearing a blue shirt.', 'A man is wearing a black shirt.', 'Nobody has a suit.', 'Nobody has a shirt on.', 'A person in a shirt.', 'A tall person in a shirt.', 'A man is wearing a white shirt.', 'A girl is on a swing.', 'The man is wearing black.', 'The boy is in the sand.', 'People standing in line.', 'Two girls on a water slide.', 'The woman is in the bathroom.', 'The child is asleep.', 'A girl is sitting.', 'The person is wearing blue jeans.', 'The dog is in the water.', 'A man jumps in the ocean.', 'The person is inside.', 'The person is on the ground.', 'Two kids are outdoors.', 'The girl is wearing a hat.', 'The man and woman are sitting at home.', 'Two kids are sleeping in bed.', 'Nobody is standing.', 'A tall person standing.', 'The man is on the moon.', 'A young man is running.', 'Two men are dancing.', 'A boy is talking on the phone.', 'The woman is wearing jeans.', 'The man is swimming at the beach.', 'There are two people.', 'The women are friends.', 'Three peopl